In [31]:
import numpy as np
import matplotlib.pyplot as plt
cm   = 1.
kg   = 1.
eV   = 1.
keV  = 10**3*eV
MeV  = 10**6*eV
s    = 1.
yr   = 365*24*60*60*s
c    = 1.

#bulk mass of Xe
m_Xe = 500*kg
#atomic number of Xe
A_Xe = 54
recoil_energy_threshold = 3*keV
#proton mass
m_p = 938.272*MeV
#observational duration
t_obs = 1*yr

#spin-independent WIMP-proton coupling cross-section
sigma_SI = 1

In [30]:
#calculate the expected 95% C.L. upper limits on the WIMP-proton coupling
#cross-sections as function of the dark matter mass

#calculate reduced mass
def mu(m_DM):
    return m_DM*m_p/(m_DM + m_p)

#calculate WIMP-proton coupling cross-section
def sigma(m_DM):
    return sigma_SI*mu(m_DM)**2*A_Xe**2/m_p**2
